In [ ]:
# -*- coding:utf-8 -*-
# DeepGlobe Images

from keras.preprocessing.image import img_to_array, load_img
import numpy as np
import glob


class dataProcess(object):
  def __init__(self, out_rows, out_cols, data_path="/content/drive/My Drive/codalab/satellite_images", label_path="/content/drive/My Drive/codalab/masks", 
               test_path="/content/drive/My Drive/codalab/test/sat", npy_path="/content/drive/My Drive/codalab/npydata", img_type="jpg"):
    self.out_rows = out_rows
    self.out_cols = out_cols
    self.data_path = data_path
    self.label_path = label_path
    self.img_type = img_type
    self.test_path = test_path
    self.npy_path = npy_path

  def create_train_data(self):
    i = 0
    print('Creating training images...')
    imgs = glob.glob(self.data_path+"/*."+self.img_type)
    labels = glob.glob(self.label_path+"/*."+self.img_type)
    imgdatas = np.ndarray((len(imgs), self.out_rows, self.out_cols, 3), dtype=np.uint8)
    imglabels = np.ndarray((len(imgs), self.out_rows, self.out_cols, 1), dtype=np.uint8)

    for x in range(len(imgs)):
      imgpath = imgs[x]
      #print('imgpath: ', imgpath)
      #labelpath_img = labels[x]
      pic_name = imgpath.split('/')[-1]
      #print(pic_name)
      #label_name = labelpath_img.split('/')[-1]
      labelpath = self.label_path + '/' + pic_name
      img = load_img(imgpath, target_size=[256, 256])
      label = load_img(labelpath, color_mode="grayscale", target_size=[256, 256])
      img = img_to_array(img)
      label = img_to_array(label)
      imgdatas[i] = img
      imglabels[i] = label
      if i % 100 == 0:
        print('Done: {0}/{1} images'.format(i, len(imgs)))
      i += 1

    print('loading done')
    np.save(self.npy_path + '/imgs_train.npy', imgdatas)
    np.save(self.npy_path + '/imgs_mask_train.npy', imglabels)
    print('Saving to .npy files done.')

  def create_test_data(self):
    i = 0
    print('Creating test images...')
    imgs = glob.glob(self.test_path + "/*." + self.img_type)
    imgdatas = np.ndarray((len(imgs), self.out_rows, self.out_cols, 3), dtype=np.uint8)
    testpathlist = []

    for imgname in imgs:
      testpath = imgname
      testpathlist.append(testpath)
      img = load_img(testpath, target_size=[256, 256]) #grayscale=False,
      img = img_to_array(img)
      imgdatas[i] = img
      i += 1

    txtname = '/content/drive/My Drive/codalab/unet_results/pic.txt'
    with open(txtname, 'w') as f:
      for i in range(len(testpathlist)):
        f.writelines(testpathlist[i] + '\n')
    print('loading done')
    np.save(self.npy_path + '/imgs_test.npy', imgdatas)
    print('Saving to imgs_test.npy files done.')

  def load_train_data(self):
    print('load train images...')
    imgs_train = np.load(self.npy_path + "/imgs_train.npy")
    imgs_mask_train = np.load(self.npy_path + "/imgs_mask_train.npy")
    imgs_train = imgs_train.astype('float32')
    imgs_mask_train = imgs_mask_train.astype('float32')
    imgs_train /= 255.
    imgs_mask_train /= 255.
    imgs_mask_train[imgs_mask_train > 0.5] = 1  
    imgs_mask_train[imgs_mask_train <= 0.5] = 0 
    return imgs_train, imgs_mask_train

  def load_test_data(self):
    print('-' * 30)
    print('load test images...')
    print('-' * 30)
    imgs_test = np.load(self.npy_path + "/imgs_test.npy")
    imgs_test = imgs_test.astype('float32')
    imgs_test /= 255.
    return imgs_test
    
if __name__ == "__main__":
    mydata = dataProcess(256, 256)
    mydata.create_train_data()
    mydata.create_test_data()

Creating training images...
Done: 0/500 images
Done: 100/500 images
Done: 200/500 images
Done: 300/500 images
Done: 400/500 images
loading done
Saving to .npy files done.
Creating test images...
loading done
Saving to imgs_test.npy files done.


In [ ]:
# -*- coding:utf-8 -*-

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import array_to_img
import cv2
#from data import *


class myUnet(object):
  def __init__(self, img_rows=256, img_cols=256):
    self.img_rows = img_rows
    self.img_cols = img_cols

  def load_data(self):
    mydata = dataProcess(self.img_rows, self.img_cols)
    imgs_train, imgs_mask_train = mydata.load_train_data()
    imgs_test = mydata.load_test_data()
    return imgs_train, imgs_mask_train, imgs_test

  def get_unet(self):
    inputs = Input((self.img_rows, self.img_cols, 3))

    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    # print(conv1)
    print("conv1 shape:", conv1.shape)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    print("conv1 shape:", conv1.shape)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    print("pool1 shape:", pool1.shape)

    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    print("conv2 shape:", conv2.shape)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    print("conv2 shape:", conv2.shape)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    print("pool2 shape:", pool2.shape)

    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    print("conv3 shape:", conv3.shape)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    print("conv3 shape:", conv3.shape)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    print("pool3 shape:", pool3.shape)

    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    print("drop4 shape:", drop4.shape)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(drop5))
    print("up6 shape:", up6.shape)
    merge6 = concatenate([drop4, up6], axis=3)
    print(up6)
    print(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    print(conv6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)
    print(conv6)
    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    print(up7)
    print(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    print(conv7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)
    print(conv7)
    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    print(up9)
    print(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    print(conv9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    print(conv9)
    conv9 = Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    print("conv9 shape:", conv9.shape)

    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)
    print(conv10)
    model = Model(inputs=inputs, outputs=conv10)

    model.compile(optimizer=Adam(lr=2e-4), loss='binary_crossentropy', metrics=['accuracy'])

    return model

  def train(self):
    print("loading data")
    imgs_train, imgs_mask_train, imgs_test = self.load_data()
    print("loading data done")
    model = self.get_unet()
    print("got unet")
    model_checkpoint = ModelCheckpoint('unet.hdf5', monitor='loss', verbose=1, save_best_only=True)
    print('Fitting model...')
    model.fit(imgs_train, imgs_mask_train, batch_size=1, epochs=100, verbose=1, validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

    print('predict test data')
    imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)
    np.save('/content/drive/My Drive/codalab/unet_results/imgs_mask_test.npy', imgs_mask_test)

  def save_img(self):
    print("array to image")
    imgs = np.load('/content/drive/My Drive/codalab/unet_results/imgs_mask_test.npy')
    piclist = []
    for line in open("/content/drive/My Drive/codalab/unet_results/pic.txt"):
      line = line.strip()
      picname = line.split('/')[-1]
      piclist.append(picname)
    for i in range(imgs.shape[0]):
      path = "/content/drive/My Drive/codalab/unet_results/" + piclist[i]
      img = imgs[i]
      img = array_to_img(img)
      img.save(path)
      cv_pic = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
      #cv_pic = cv2.resize(cv_pic,(256,256),interpolation=cv2.INTER_CUBIC)
      binary, cv_save = cv2.threshold(cv_pic, 127, 255, cv2.THRESH_BINARY)
      cv2.imwrite(path, cv_save)

if __name__ == '__main__':
    myunet = myUnet()
    model = myunet.get_unet()
    # model.summary()
    # plot_model(model, to_file='model.png')
    myunet.train()
    myunet.save_img()

conv1 shape: (None, 256, 256, 64)
conv1 shape: (None, 256, 256, 64)
pool1 shape: (None, 128, 128, 64)
conv2 shape: (None, 128, 128, 128)
conv2 shape: (None, 128, 128, 128)
pool2 shape: (None, 64, 64, 128)
conv3 shape: (None, 64, 64, 256)
conv3 shape: (None, 64, 64, 256)
pool3 shape: (None, 32, 32, 256)
drop4 shape: (None, 32, 32, 512)
up6 shape: (None, 32, 32, 512)
Tensor("conv2d_10/Relu:0", shape=(None, 32, 32, 512), dtype=float32)
Tensor("concatenate/concat:0", shape=(None, 32, 32, 1024), dtype=float32)
Tensor("conv2d_11/Relu:0", shape=(None, 32, 32, 512), dtype=float32)
Tensor("conv2d_12/Relu:0", shape=(None, 32, 32, 512), dtype=float32)
Tensor("conv2d_13/Relu:0", shape=(None, 64, 64, 256), dtype=float32)
Tensor("concatenate_1/concat:0", shape=(None, 64, 64, 512), dtype=float32)
Tensor("conv2d_14/Relu:0", shape=(None, 64, 64, 256), dtype=float32)
Tensor("conv2d_15/Relu:0", shape=(None, 64, 64, 256), dtype=float32)
Tensor("conv2d_19/Relu:0", shape=(None, 256, 256, 64), dtype=float32)